# Projet 3 (LU3IN005) : Chaines de Markov et épidémiologie :
# Propagation d’une épidémie dans une population

### <u>Groupe 6 :</u>
<pr>Samy (Adem) YAZID<br>Numéro étudiant : $3811684$<br><br>Margaux MARSELOO<br>Numéro étudiant : $28630611$</pr> 

### <u>**Consignes :**</u>
<pr>

L’objectif de ce projet est de manipuler des chaînes de Markov pour étudier la propagation d’une épidémie dans une population.<br>
Notre rendu est un notebook contenant les codes commentés et les résultats interprétés.<br>
Les packages <em>numpy</em>, <em>random</em> et <em>matplotlib</em> sont utilisés.<br>
Nous allons étudier des populations constituées de $3$ types d’individus.<br>
Chaque individu est dans un des $3$ états suivant : sain $S$, infecté $I$ ou guéri $R$.
</pr>

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random as rd

def rec_extract_values(iterable, dico):
    """
    Paramètres:
    iterable : any
        Iterable (ou non) dont on veut extraire les valeurs dans un dictionnaire
    dico : dict
        Dictionnaire qui stockera les valeurs extraites
        
    Retourne:
    None

    Description:
    Fonction permettant d'extraire toutes les valeurs de iterable en les stockant
    dans dico, et qui compte toutes les fois où chaque valeur est apparue.
    (Dans cette fonction les strings sont considérés comme non-iterable, donc
    seront compté comme une valeur quelconque)
    
    Exemple:
    dico : {}
    rec_extract_values([1, 3, [3, 7]])
    dico : {1:1, 3:2, 7:1}
    """
    if type(iterable) in [type([]), type(np.matrix([])), type(np.array([]))]:
        for l in iterable:
            rec_extract_values(l, dico)
        return dico
    else:
        if iterable not in dico:
            dico[iterable] = 1
        else:
            dico[iterable] += 1
            
def rec_sum_values(iterable):
    """
    Paramètres:
    iterable : any iterable type of (int or float)
        Iterable dont on veut sommer les valeurs
        
    Retourne:
    somme : int or float
        Somme des valeurs de iterable
    
    Description:
    Fonction permettant de retourner la somme de toutes les valeurs de iterable.
    
    Exemple:
    rec_sum_values([1, 3, [3, 7.5]]) : 14.5
    """
    if type(iterable) in [type([]), type(np.matrix([])), type(np.array([]))]:
        somme = 0
        for l in iterable:
            somme += rec_sum_values(l)
        return somme
    else:
        return iterable
    
def dict_zero(dico):
    """
    Paramètres:
    dico : dict
        Dictionnaire dont on veut mettre toutes ses valeurs à 0
        
    Retourne:
    None
    
    Description:
    Fonction permettant pour un dictionnaire de conserver ses clés tout en fixant leur valeur à 0.
    
    Exemple:
    dico : {1:1, 3:2, 7:1}
    dict_zero(dico)
    dico : {1:0, 3:0, 7:0}
    """
    for key in dico:
        dico[key] = 0

## I) <u>Apprentissage des paramètres d’un modèle à partir de données</u>

<pr>
    
Dans notre modèle, nous allons considérer qu’à chaque temps :
- Chaque individu sain peut rester sain ou devenir infecté
- Chaque individu infecté peut rester infecté ou devenir guéri
- Chaque individu guéri reste guéri
et que la probabilité de passer d’un état à l’autre ne dépend que de l’état précédent.

Nous disposons d’une séquence d’observations et nous souhaitons apprendre les paramètres de la
chaîne de Markov permettant de modéliser le processus sous-jacent qui a généré la séquence.<br>
Nous avons suivi un individu pendant $10$ jours, afin de déterminer a chaque temps dans quel état se trouvait l’individu.<br>
Nous avons obtenu la séquence d’observation suivante : $S, S, S, I, I ,I ,I , I, I, R$.
</pr>


<pr><u>**Question 1:**</u> À partir de cette séquence d’observation, estimez de les probabilités de transition entre les états et dresser la matrice de probabilité de transitions.</pr>


Matrice de transition du modèle :
$
\begin{pmatrix}
  &   S &   I &   R \\
S & 2/3 & 1/3 &   0 \\
I &   0 & 5/6 & 1/6 \\
R &   0 &   0 &   1 \\
\end{pmatrix}
$

In [ ]:
matrice_transition = np.array([[2/3, 1/3,   0], 
                               [  0, 5/6, 1/6], 
                               [  0,   0,   1]])

<pr>
Nous avons ensuite suivi une population de $5000$ individus, pendant $200$ jours.<br>
Pour lire les données, nous avons utiliser <code>np.loadtxt("data_exo_2022.txt")</code>.<br>
Les individus sains sont notés $0$, les infectés $1$ et les guéris $2$.

<u>**Question 2:**</u><br>
    
1) Lire des données
    
</pr>

In [ ]:
data = np.loadtxt("data_exo_2022.txt")

<pr>
    
2) Estimez les probabilités de transition entre les états et dressez la matrice de probabilité de transitions
(indice pour vérifier vos résultats : la première ligne de la matrice est $[0.9308, 0.0691, 0.]$).

</pr>

In [ ]:
def analyse_seq(sequence, nb_etats=-1):
    sequence = sequence.astype(int)
    
    if nb_etats == -1:
        card = {elem : 0 for elem in set(sequence)}
        nb_etats = len(card)
        mat_transition_etats = np.zeros((nb_etats, nb_etats))
    else:
        mat_transition_etats = np.zeros((nb_etats, nb_etats))
    
    #on détermine la fréquence
    for etat in range(1, len(sequence)):
        mat_transition_etats[sequence[etat-1]][sequence[etat]] += 1
    mat_transition_etats[sequence[etat]][sequence[etat]] += 1
        
    return mat_transition_etats  

def matrice_proba_transition_liste(liste_sequence):
    card = {}
    rec_extract_values(liste_sequence, card)
    matrice_finale = np.zeros((len(card), len(card)))
    
    for seq in liste_sequence:
        matrice_finale += analyse_seq(seq, len(card))
    print("\nMatrice des fréquences :")
    print(matrice_finale)
    print("\n")
    print("card[0] = ",card[0])
    for i in range(len(matrice_finale)):
        for j in range(len(matrice_finale[0])):
            matrice_finale[i][j] /= card[i]
            
    return matrice_finale

print("Matrice de transition de l'exemple:\n")
print(matrice_proba_transition_liste([np.array([0., 0., 0., 1., 1., 1., 1., 1., 1., 2.])]))
print("\nMatrice de transition des 5000 individus:\n")
print(matrice_proba_transition_liste(data))

Matrice de fréquence finale:
$
\begin{pmatrix}
  &   S &   I &   R \\
S & 274506 & 20408 &   0 \\
I &   0 & ? & ? \\
R &   0 &   0 &  0 \\
\end{pmatrix}
$

## II) <u>Description du premier modèle</u>
<pr>La probabilité pour un individu d’être dans un de ces $3$ états au temps $t$, ne dépend que l’état dans
lequel il est au temps $t − 1$.<br>
Un individu dans l’état sain a une probabilité de $0.92$ de rester sain et une probabilité de $0.08$ de
devenir infecté. Si l’individu est infecté, il peut le rester avec une probabilité de $0.93$ et être guéri avec
une probabilité de $0.07$. S’il est dans l’état guéri, il reste dans cet état avec une probabilité de $1$.

<u>**Question 1:**</u> À partir du graphe de transition, créez la matrice de transition $A$, la matrice contenant les probabilités de transition entre les différents états. Créez une fonction permettant de vérifier qu’une matrice est stochastique.
</pr>

Matrice de transition du premier modèle :<br>
$A = 
\begin{pmatrix}
  &    S &    I &    R \\
S & 0.92 & 0.08 &    0 \\
I &    0 & 0.93 & 0.07 \\
R &    0 &    0 &    1 \\
\end{pmatrix}
$

In [ ]:
matrice_transition_modele1 = np.array([[0.92, 0.08,    0], 
                                       [   0, 0.93, 0.07], 
                                       [   0,    0,    1]])

def est_stochastique(matrice):
    epsilon = 0.00001
    for ligne in matrice:
        if abs(rec_sum_values(ligne) - 1) >= epsilon:
            return False
    return True

print("La matrice de transition de la partie I) est-elle stochastique ? :", est_stochastique(matrice_transition))

print("La matrice de transition de la partie II) est-elle stochastique ? :", est_stochastique(matrice_transition))

<pr>
Au temps $t = 0$, un individu a une probabilité de $0.9$ d’être sain et $0.1$ d’être infecté.

<u>**Question 2:**</u> Créez $π_0$ la distribution de probabilité initiale.
</pr>

$
π_0 = 
\begin{pmatrix}
   S &    I &    R \\
0.90 & 0.10 &    0 \\
\end{pmatrix}
$

In [ ]:
pi_0 = np.array([0.90,
                 0.10,
                 0.00])

<pr>
Nous allons étudier l’évolution du nombre d’individu sains, infectés et guéris au cours du temps.
Dans un premier temps nous étudierons la distribution théorique, puis la distribution observée sur des simulations.
</pr>

### <u>Distribution $π_t$<u>

<pr><u>**Question 1:**</u> En utilisant $π_0$ et $A$, donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 1$ (faire d’abord le calcul à la main).
</pr>


$
π_1 = π_0 * A =
\begin{pmatrix}
   S &    I &    R \\
0.90*0.92 + 0*0 & 0.90*0.08 + 0.10*0.93 &    0.10*0.07 + 0*1 \\
\end{pmatrix}
=
\begin{pmatrix}
   S &    I &    R \\
0.828 & 0.165 & 0.007\\
\end{pmatrix}
$

In [ ]:
pi_1 = np.dot(pi_0, matrice_transition_modele1) 

print("pi_1 =", pi_1)

<pr><u>**Question 2:**</u> Donnez la probabilité pour un individu d’être sain, infecté ou guéri au temps $t = 2$ (faire d’abord le calcul à la main).
</pr>

$
π_2 = π_1 * A =
\begin{pmatrix}
   S &    I &    R \\
0.828*0.92 + 0.007*0   &   0.828*0.08 + 0.165*0.93   &   0.165*0.07 + 0.007*1 \\
\end{pmatrix}
=
\begin{pmatrix}
   S &    I &    R \\
0.76176 & 0.21969 & 0.01855\\
\end{pmatrix}
$

In [ ]:
pi_2 = [0.828*0.92 + 0.007*0   ,
        0.828*0.08 + 0.165*0.93,
        0.165*0.07 + 0.007*1   ]

print("pi_2 =", pi_2)

pi_2 = np.dot(pi_1, matrice_transition_modele1) 

print("pi_2 =", pi_2)


<pr><u>**Question 3:**</u> De même pour chaque temps $t$ entre $1$ et $200$, calculez la distribution théorique des effectifs dans chaque état (<em>Rappel:</em> $π_{t+1} = π_t . A$ ).
</pr>

In [ ]:
"""
def liste_pi(pi_0, A, n):
    l_pi = [pi_0]
    for i in range(n):
        pi_i = []
        for j in range(len(pi_0)):
            pi_i.append(l_pi[-1][j] * A[j][j] + l_pi[-1][j-1] * A[j-1][j])
        l_pi.append(pi_i)
    return l_pi
"""

def liste_pi(pi_0, A, n):
    l_pi = [pi_0]
    for i in range(1, n+1):
        l_pi.append(np.dot(l_pi[i-1], matrice_transition_modele1))
    return l_pi

liste_pi_i = np.array(liste_pi(pi_0, matrice_transition_modele1, 200))
print("Liste des pi_i pour i entre 0 et 200 :")
print(liste_pi_i)

<pr><u>**Question 4:**</u> Représentez graphiquement la probabilité d’être dans chaque état en fonction du temps. (+ décrivez un peu ce que vous observez).
</pr>

In [ ]:
def affiche_graphe_multi(liste_pi):
    #on crée les ordonnées
    liste_sains = []
    liste_infectes = []
    liste_gueris = []
    for pi in liste_pi_i:
        liste_sains.append(pi[0])
        liste_infectes.append(pi[1])
        liste_gueris.append(pi[2])
    
    #on crée les abscisses
    abscisses = [i for i in range(len(liste_pi))]
    
    #on plot le graphe comme une courbe
    plt.plot(abscisses, liste_sains)
    plt.plot(abscisses, liste_infectes)
    plt.plot(abscisses, liste_gueris)
    
    #on légende le graphe
    plt.legend(["Personnes saines", "Personnes infectées", "Personnes guéries"], loc='center right')
    
    #on nomme l'axe des abscisses
    plt.xlabel("Temps t")
    #on nomme l'axe des ordonnées
    plt.ylabel("Probabilités")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Graphique des proportions la population en fonction du temps")
       
    #on affiche le graphe
    plt.show()

affiche_graphe_multi(liste_pi_i)

<img src="Graph_Partie2.png"  align="bottom">

<pr><u>**Interprétation du graphique:**</u>
    Nous remarquons qu'il n'y qu'une seule vague d'infection. En effet, les états sains et infectés sont transients tant dit que l'état guéri est récurrent. L'état guéri est en effet un état absorbant.Nous voyons bien ici que ce modèle converge vers <code>π* = [ 0.0 , 0.0 , 1. ]</code>.<br>
</pr>

### <u>Tirage aléatoire des états</u>
<pr>Vous allez générer une séquence de taille $T$ en utilisant cette chaîne de Markov. Pour générer une séquence aléatoire, choisissiez un état initial au hasard (en utilisant $π_0$) ; puis choisissez les états suivants en suivant les probabilités de transition <em>(= la matrice de transition $A$).</em>
Vous pouvez prendre $T=150$.
</pr>

In [14]:
def genere_sequence(pi_0, A, n):
    p = rd.random()
    sequence = []
    for i in range(len(pi_0)):
        if p < sum(pi_0[:i+1]):
            sequence.append(i)
            break
    for i in range(1, n):
        p = rd.random()
        for j in range(len(A)):
            if p < sum(A[sequence[-1]][:j+1]):
                sequence.append(j)
                break
    return np.array(sequence)

print("La séquence aléatoire de taille 150 suivant cette chaîne de Markov est :\n")
print(genere_sequence(pi_0, matrice_transition_modele1, 150))

La séquence aléatoire de taille 150 suivant cette chaîne de Markov est :

[0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### <u>Modélisation d’une population</u>
<pr>Vous avez généré une séquence d’état pour un individu. Maintenant vous allez générer un ensemble de séquences pour une population de 200 individus.

<u>**Question 1:**</u> À chaque temps $t$, comptez le nombre d’individus sains, infectés et guéris dans la population et affichez l’évolution du nombre d’individus dans les trois états en fonction du temps.
</pr>

In [15]:
def genere_liste_sequence(pi_0, A, nb_individu, temps):
    liste_sequence = []
    for i in range(nb_individu):
        liste_sequence.append(genere_sequence(pi_0, A, temps))
    return np.array(liste_sequence)

def comptage_liste_sequence(liste_sequence, nb_etats=3):
    liste_comptage = []
    for jour in range(len(liste_sequence[0])):
        comptage = [0 for i in range(nb_etats)]
        for seq in liste_sequence:
            comptage[seq[jour]] += 1
        liste_comptage.append(comptage)
    return np.array(liste_comptage)

liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 200, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 200 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 200 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 200 séquences est :

[[0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 ...
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [1 1 1 ... 2 2 2]]


La liste de 200 séquences compte cet effectif au cours du temps:

[[183  17   0]
 [167  31   2]
 [152  43   5]
 [141  53   6]
 [128  62  10]
 [117  70  13]
 [111  68  21]
 [105  72  23]
 [ 98  70  32]
 [ 89  74  37]
 [ 83  78  39]
 [ 74  83  43]
 [ 69  82  49]
 [ 65  80  55]
 [ 57  79  64]
 [ 50  78  72]
 [ 46  77  77]
 [ 45  72  83]
 [ 41  74  85]
 [ 35  72  93]
 [ 32  71  97]
 [ 29  64 107]
 [ 28  60 112]
 [ 26  57 117]
 [ 24  56 120]
 [ 23  53 124]
 [ 20  54 126]
 [ 19  52 129]
 [ 17  48 135]
 [ 17  46 137]
 [ 16  46 138]
 [ 15  47 138]
 [ 14  45 141]
 [ 13  44 143]
 [ 12  42 146]
 [ 12  37 151]
 [ 12  34 154]
 [ 12  32 156]
 [ 12  31 157]
 [  9  32 159]
 [  9  31 160]
 [  7  30 163]
 [  7  27 166]
 [  7  26 167]
 [  6  24 170]
 [  6  22 172]
 [  6  21 173]
 [  6  20 174]
 [  6  20 174]
 [  6  19 175]
 [  6  16 178]
 [  6  16 178]

<pr><u>**Question 2:**</u> Affichez le pourcentage d’individus sains infectés et guéris en fonction du temps.
</pr>

In [16]:
def pourcentage_liste_sequence(liste_sequence):
    liste_pourcentage = [[((cpt/sum(l_cpt))*100) for cpt in l_cpt] for l_cpt in comptage_liste_sequence(liste_sequence)]
    return np.array(liste_pourcentage)

liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 200 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[ 91.5   8.5   0. ]
 [ 83.5  15.5   1. ]
 [ 76.   21.5   2.5]
 [ 70.5  26.5   3. ]
 [ 64.   31.    5. ]
 [ 58.5  35.    6.5]
 [ 55.5  34.   10.5]
 [ 52.5  36.   11.5]
 [ 49.   35.   16. ]
 [ 44.5  37.   18.5]
 [ 41.5  39.   19.5]
 [ 37.   41.5  21.5]
 [ 34.5  41.   24.5]
 [ 32.5  40.   27.5]
 [ 28.5  39.5  32. ]
 [ 25.   39.   36. ]
 [ 23.   38.5  38.5]
 [ 22.5  36.   41.5]
 [ 20.5  37.   42.5]
 [ 17.5  36.   46.5]
 [ 16.   35.5  48.5]
 [ 14.5  32.   53.5]
 [ 14.   30.   56. ]
 [ 13.   28.5  58.5]
 [ 12.   28.   60. ]
 [ 11.5  26.5  62. ]
 [ 10.   27.   63. ]
 [  9.5  26.   64.5]
 [  8.5  24.   67.5]
 [  8.5  23.   68.5]
 [  8.   23.   69. ]
 [  7.5  23.5  69. ]
 [  7.   22.5  70.5]
 [  6.5  22.   71.5]
 [  6.   21.   73. ]
 [  6.   18.5  75.5]
 [  6.   17.   77. ]
 [  6.   16.   78. ]
 [  6.   15.5  78.5]
 [  4.5  16.   79.5]
 [  4.5  15.5  80. ]
 [  3.5  15.   81.5]
 [  3.5  13.5 

<pr><u>**Question 3:**</u> Quand $t$ est grand, quelle est la proportion d’individus sains, infectés et guéris ?
</pr>

<pr>
Lorsque $t$ est grand, la répartition de la population est la suivante :
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
</pr>

<pr><u>**Question 4:**</u> Refaites les questions précédentes avec des populations de tailles différentes, $5$ individus et $5000$ individus par exemple.
</pr>

In [17]:
#Pour 5 individus
liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 5, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 5 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 5 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 5 séquences est :

[[1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2]
 [0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

In [18]:
liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 5 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 5 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[ 80.  20.   0.]
 [ 80.  20.   0.]
 [ 60.  40.   0.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 60.  20.  20.]
 [ 40.  40.  20.]
 [ 40.  40.  20.]
 [ 40.  20.  40.]
 [ 20.  40.  40.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.  20.  60.]
 [ 20.   0.  80.]
 [  0.  20.  80.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.]
 [  0.   0. 100.

<pr>
Lorsque $t$ est grand, la répartition de la population est, comme précédemment, la suivante :
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
</pr>

In [19]:
#Pour 5000 individus
liste_sequence = genere_liste_sequence(pi_0, matrice_transition_modele1, 5000, 150)
liste_comptage = comptage_liste_sequence(liste_sequence)

print("La liste de 5000 séquences est :\n")
print(liste_sequence)

print("\n\nLa liste de 5000 séquences compte cet effectif au cours du temps:\n")
print(liste_comptage)

La liste de 5000 séquences est :

[[0 1 1 ... 2 2 2]
 [1 1 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 ...
 [0 0 1 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]]


La liste de 5000 séquences compte cet effectif au cours du temps:

[[4523  477    0]
 [4118  842   40]
 [3787 1105  108]
 [3502 1313  185]
 [3232 1496  272]
 [2980 1658  362]
 [2737 1793  470]
 [2498 1894  608]
 [2300 1958  742]
 [2112 2010  878]
 [1933 2039 1028]
 [1800 2036 1164]
 [1643 2049 1308]
 [1521 2063 1416]
 [1419 1992 1589]
 [1321 1956 1723]
 [1203 1936 1861]
 [1111 1885 2004]
 [1021 1843 2136]
 [ 939 1787 2274]
 [ 870 1733 2397]
 [ 809 1682 2509]
 [ 742 1616 2642]
 [ 678 1579 2743]
 [ 629 1507 2864]
 [ 575 1451 2974]
 [ 534 1396 3070]
 [ 493 1329 3178]
 [ 452 1276 3272]
 [ 419 1232 3349]
 [ 381 1174 3445]
 [ 346 1137 3517]
 [ 323 1088 3589]
 [ 303 1045 3652]
 [ 280 1000 3720]
 [ 261  958 3781]
 [ 240  904 3856]
 [ 218  883 3899]
 [ 193  837 3970]
 [ 176  812 4012]
 [ 163  765 4072]
 [ 154  725 4121]
 [ 145  692 4163]
 [ 

In [20]:
liste_pourcentage = pourcentage_liste_sequence(liste_sequence)
print("\n\nLa liste de 5000 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:")
print(liste_pourcentage)



La liste de 5000 séquences compte ces pourcentage d'individus sains/infectés/guéris au cours du temps:
[[9.046e+01 9.540e+00 0.000e+00]
 [8.236e+01 1.684e+01 8.000e-01]
 [7.574e+01 2.210e+01 2.160e+00]
 [7.004e+01 2.626e+01 3.700e+00]
 [6.464e+01 2.992e+01 5.440e+00]
 [5.960e+01 3.316e+01 7.240e+00]
 [5.474e+01 3.586e+01 9.400e+00]
 [4.996e+01 3.788e+01 1.216e+01]
 [4.600e+01 3.916e+01 1.484e+01]
 [4.224e+01 4.020e+01 1.756e+01]
 [3.866e+01 4.078e+01 2.056e+01]
 [3.600e+01 4.072e+01 2.328e+01]
 [3.286e+01 4.098e+01 2.616e+01]
 [3.042e+01 4.126e+01 2.832e+01]
 [2.838e+01 3.984e+01 3.178e+01]
 [2.642e+01 3.912e+01 3.446e+01]
 [2.406e+01 3.872e+01 3.722e+01]
 [2.222e+01 3.770e+01 4.008e+01]
 [2.042e+01 3.686e+01 4.272e+01]
 [1.878e+01 3.574e+01 4.548e+01]
 [1.740e+01 3.466e+01 4.794e+01]
 [1.618e+01 3.364e+01 5.018e+01]
 [1.484e+01 3.232e+01 5.284e+01]
 [1.356e+01 3.158e+01 5.486e+01]
 [1.258e+01 3.014e+01 5.728e+01]
 [1.150e+01 2.902e+01 5.948e+01]
 [1.068e+01 2.792e+01 6.140e+01]
 [9.

<pr>
Lorsque $t$ est grand, la répartition de la population est, encore une fois, la suivante :
$ 
\begin{pmatrix}
   S &    I &     R \\
 0\% &  0\% & 100\% \\
\end{pmatrix}
$
</pr>

### <u>Longueur de l’infection</u>
<pr>

<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $I$.
</pr>

In [21]:
def longueur_moyenne(liste_sequence, categorie):
    moy = 0
    for seq in liste_sequence:
        for etat in seq:
            if etat == categorie:
                moy += 1
    return moy/len(liste_sequence)

longueur_moyenne_infectes = longueur_moyenne(liste_sequence, 1)

print("La longueur moyenne d'une séquence de I est de", longueur_moyenne_infectes, "jours")

La longueur moyenne d'une séquence de I est de 14.4158 jours


<u>**Question 2:**</u> Afficher la distribution observée de la longueur d’infection (peut-être que pour représenter une distribution, ça serait une bonne idée de faire un histogramme).
</pr>

In [26]:
def repartition_longueur_moyenne(liste_sequence, categorie):
    dico_repart = {}
    for seq in liste_sequence:
        cpt = 0
        for etat in seq:
            if etat == categorie:
                cpt += 1
        if cpt in dico_repart:
            dico_repart[cpt] += 1
        else:
            dico_repart[cpt] = 1
    for i in range(max(dico_repart)):
        if i not in dico_repart:
            dico_repart[i] = 0
    
    return dico_repart

def affiche_graphe_repartition(dico_repart):
    #on crée les abscisses
    abscisses = [key for key in dico_repart]
    abscisses.sort()
    
    #on crée les ordonnées
    ordonnees = [dico_repart[key] for key in abscisses]
    
    #on plot l'histogramme
    plt.bar(abscisses, ordonnees)
    
    #on nomme l'axe des abscisses
    plt.xlabel("Longueur de l'infection (en jours)")
    #on nomme l'axe des ordonnées
    plt.ylabel("Nombre de séquences")
    
    #on paramètre l'échelle des abscisses suivant l'option x_scale
    plt.xscale("linear")
    #on paramètre l'échelle des ordonnées comme linéaire
    plt.yscale("linear")
    
    #on donne un titre au graphique
    plt.title("Histogramme de la répartition de la longueur de l'infection")
       
    #on affiche le graphe
    plt.show()

dico_repart = repartition_longueur_moyenne(liste_sequence, 1)

#affiche_graphe_repartition(dico_repart)

<img src="Histo_Partie2.png"  align="bottom">

<u>**Question 3:**</u> Calculez la longueur théorique d’une séquence de $I$ (vous pourrez utiliser l’espérance de la loi géométrique en justifiant).
</pr>

In [23]:
p = matrice_transition_modele1[1][2]
print("L'espérance de la longueur d'une séquence de I est", (1/p))

L'espérance de la longueur d'une séquence de I est 14.285714285714285


<u>**Question 4:**</u> Comparer la longueur estimée et la longueur théorique.
</pr>

<pr>
La longueur estimée est de $14.23$ jours, tandis que la longueur théorique est de $14.29$ jours.<br>
On voit donc que les deux valeurs coïncident totalement.
</pr>

<u>**Question 5:**</u> Affichez la distribution théorique de la longueur d’infection.
</pr>

### <u>Petites modifications autour de ce premier modèle</u>
<pr>Vous pouvez maintenant modifier le modèle pour étudier différents cas de figure, en
faisant varier certains paramètres :
</pr>

<u>**Question 1:**</u> Faire varier la taille de la population.
</pr>

<u>**Question 2:**</u> Faire varier la distribution de probabilité initiale.
</pr>

<u>**Question 3:**</u> Faire varier les probabilités de transition.
</pr>

## III) <u>Description du second modèle</u>
<pr>Nous allons maintenant considérer un second modèle, les individus guéris peuvent redevenir sains avec une probabilité de $0.02$. Ils-elles peuvent perdre leur immunité face à la maladie.
</pr>

### <u>Analyse du modèle</u>

<pr>

<u>**Question 1:**</u> Ce processus peut-il être modélisé par une chaîne de Markov ?
</pr>

<pr>

<u>**Question 2:**</u>
Donnez la nouvelle matrice de transition.
</pr>

Matrice de transition du second modèle :<br>
$A = 
\begin{pmatrix}
  &    S &    I &    R \\
S & 0.92 & 0.08 &    0 \\
I &    0 & 0.93 & 0.07 \\
R & 0.02 &    0 & 0.98 \\
\end{pmatrix}
$

In [24]:
mat_transition_modele2 = np.array([[0.92, 0.08,    0], 
                                   [   0, 0.93, 0.07], 
                                   [0.02,    0, 0.98]])

<pr>
    
<u>**Question 3:**</u> Quels est la nature des états de cette chaîne de Markov ? Est-elle périodique ? Est-elle irréductible ?
</pr>

<pr>
    
<u>**Question 4:**</u> Calculez la matrice $A × A$. À quoi correspond-elle ? Est elle stochastique ? Même question pour $A^3$ et $A^4$.
</pr>

<pr>
$
A^2 = 
\begin{pmatrix}
  &      S &      I &      R \\
S & 0.8464 &  0.148 & 0.0056 \\
I & 0.0014 & 0.8649 & 0.1337 \\
R &  0.038 & 0.0016 & 0.9604 \\
\end{pmatrix}
$
<br><br>
$
A^3 = 
\begin{pmatrix}
  &        S &        I &        R \\
S &   0.7788 & 0.205352 & 0.191569 \\
I & 0.003962 & 0.804469 &     0.07 \\
R & 0.054168 & 0.004528 & 0.941304 \\
\end{pmatrix}
$
<br><br>
$
A^4 = 
\begin{pmatrix}
  &          S &          I &          R \\
S & 0.71681296 & 0.25328136 & 0.02990568 \\
I & 0.00747642 & 0.74847313 & 0.24405045 \\
R & 0.06866064 & 0.00854448 & 0.92279488 \\
\end{pmatrix}
$
</pr>

In [25]:
A_pow_2 = np.matmul(mat_transition_modele2, mat_transition_modele2)
A_pow_3 = np.matmul(A_pow_2, mat_transition_modele2)
A_pow_4 = np.matmul(A_pow_3, mat_transition_modele2)

print("La matrice A^2 est-elle stochastique ? :", est_stochastique(A_pow_2))
print("La matrice A^3 est-elle stochastique ? :", est_stochastique(A_pow_3))
print("La matrice A^4 est-elle stochastique ? :", est_stochastique(A_pow_4))

La matrice A^2 est-elle stochastique ? : True
La matrice A^3 est-elle stochastique ? : True
La matrice A^4 est-elle stochastique ? : True


<pr>
    
<u>**Question 5:**</u> Réalisez les nouvelles simulations. Comment la population évolue-t-elle ?
</pr>

<pr>
    
<u>**Question 6:**</u> Refaites les simulations avec une autre distribution de probabilité initiale (par exemple si au temps $t = 0$, nous avons $90%$ d’infectés et $10%$ de sains). Explorez d’autres initialisations et comme pour chaque question commentez vos observations.
</pr>

<pr>
    
<u>**Question 7:**</u> Calculez la distribution de probabilité stationnaire et comparez ce résultat avec les simulations pour
$t$ assez grand.
</pr>

### <u>Longueur de l’immunité</u>

<pr>
    
On peut se demander combien de temps un individu qui a été malade, reste protégé contre la maladie.
    
<u>**Question 1:**</u> À partir des simulations, estimez la longueur moyenne d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

<pr>
    
<u>**Question 3:**</u> Affichez la distribution théorique et la distribution observée de la longueur de l’immunité.
</pr>

### <u>Modifier le modèle</u>

<pr>
   
<u>**Question 1:**</u> Comment l’épidémie évolue-t-elle si vous modifiez la probabilité pour un individu sain de devenir infecté ? Quelle est la nouvelle distribution a l’équilibre ?
</pr>

<pr>
    
<u>**Question 2:**</u> Calculez la longueur théorique d’une séquence de $R$.
</pr>

## IV) <u>Confinement</u>
<pr>On peut imaginer que si des mesures de distanciation sociale sont mises en place, la probabilité de devenir infecté devient nulle.

Nous allons alterner entre les périodes de non distanciation et de distanciation :
- En période de non-confinement, nous utilisons la matrice de transition de l’exercice précédent
- En période de confinement, la probabilité de devenir infecté pour un individu sain devient nulle

<u>**Question 1:**</u> Commencez les simulations avec la matrice de transition de l’exercice précédent.
- On peut considérer qu’au temps initial tous les individus sont sains.
- Quand il y a $25%$ d’individus infectés dans la population, nous passons en période de distanciation. Le nombre d’individus infectés va décroître.
- Quand il y a moins de $10%$ d’infectés, le confinement est levé.
</pr>

<pr>
    
<u>**Question 2:**</u> Faites les simulations pour une population assez grande, représentez l’évolution du nombre d’individus à chaque temps $t$ (vous devriez voir des “vagues”), et notez les temps de confinement et de deconfinement.
</pr>

<pr>
    
<u>**Question 3:**</u> Combien de confinements / déconfinements sont nécessaires ?
</pr>

## V) <u>Discussion</u>

<pr>

<u>**Question 1:**</u> Quelles remarques critiques pouvez faire sur les modèles utilisés ?
</pr>

<pr>
    
<u>**Question 2:**</u> <em>(Optionnelle)</em> Proposez des améliorations et implémentez-les si possible.
</pr>